#**DATA PREPARATION 2**

**Goal** : Preprocessing the data to be "machine learning" ready

**Steps**
1. Importing data prepared from data preparation 1 along withe the necessary libraries
2. Splitting the data set from onset to avoid bias
3. Converting all dtytpes into numeric dtypes
4. Handling missing values
5. Selecting relevant columns from ML data
6. Outlier detection
7. Handling data imbalance using SMOTE

## Computing the descriptors

In [ ]:
!pip install rdkit
!pip install mordred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 35.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.9 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=8615615113ee07ddf4081e9f627b79973afdf991335f09140edbfbe7cc2c9ce4
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import numpy as np


url = "https://raw.githubusercontent.com/Ghaby-X/VP16-inhibitors-pred/main/Assayresults_preprocessed.csv"

In [ ]:
df = pd.read_csv(url)
df

,Unnamed: 0,PUBCHEM_CID,Canonical_smiles,Activity_score,Activity_outcome
0,0,72810.0,C1=CC2=C(C3=C(C=CC=N3)C(=O)C2=O)N=C1,100.0,1
1,1,23618032.0,CN(C)C1=NC=NC2=C1N=CN2[C@H]3[C@@H]([C@@H]([C@H...,99.0,1
2,2,99927.0,CC1C(C(CC(O1)OC2CC(OC(C2O)C)OC3=CC4=CC5=C(C(=O...,99.0,1
3,3,254021.0,C1=CC=C2C(=C1)C3=C(C=C(C=C3)N)C(=O)C2=O,99.0,1
4,4,45280821.0,CCC(C)C1C(CC(=O)O[C@H](C(=O)[C@H](C(=O)NC(C(=O...,99.0,1
...,...,...,...,...,...
2333,2337,665706.0,C1CC2=C(C1)NN=C2C3=NN=C4N3N=C(CS4)C5=CC=CC=C5,0.0,0
2334,2338,25015463.0,CN1CCC(CC1)NC(=O)C2=CC(=C(C=C2)NC3=NC=C4C(=N3)...,0.0,0
2335,2339,667208.0,CC1=CC2=C(C3=C(CCC3)C(=O)O2)C(=C1)OC(C)C(=O)OC,0.0,0
2336,2340,42628533.0,CN1CCC(CC1)NC(=O)C2=CC(=C(C=C2)NC3=NC=C4C(=N3)...,0.0,0


In [ ]:
from rdkit import Chem
from mordred import Calculator, descriptors

ModuleNotFoundError: ignored

In [ ]:
smiles_list = df['Canonical_smiles'].to_list()  # convert the Canonical Smiles columns to a list
mols = [Chem.MolFromSmiles(smi) for smi in smiles_list] # generates the 2D structures of molecular smiles

In [ ]:
#Function to calculate mordred descriptors
def mordred_desc(data):  #receives canonical smiles in a list datastructure
  calc = Calculator(descriptors, ignore_3D=True)
  return calc.pandas(data)

In [ ]:
df_mordred = mordred_desc(mols)
df_mordred

In [ ]:
df1 = df_mordred.copy()

In [ ]:
df1.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('O'), dtype('bool')],
      dtype=object)

In [ ]:
#function that checks if an entry dtype is numeric. If not, it converts it to numeric or not available

def float_converter(x):
    if not isinstance(x, (float, int, bool)):
        try:
            return float(x)
        except ValueError:
            return np.nan
    else:
        return x


# Apply the function to all columns in the DataFrame


In [ ]:
df2 =  df1.applymap(float_converter).copy()
df2

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,12.704061,10.489183,0,0,21.679248,2.500522,5.001045,21.679248,1.354953,3.734854,...,9.975203,49.287122,210.042927,9.547406,376,29,88.0,108.0,4.666667,3.500000
1,26.345939,21.039992,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,10.475737,87.611973,507.199695,7.803072,3400003745,57,180.0,216.0,NaN,7.583333
2,59.797706,41.739763,0,0,95.209049,2.573149,5.146298,95.209049,1.252751,5.260938,...,11.452400,116.888499,1084.472653,7.134689,32423,148,416.0,505.0,29.888889,16.333333
3,13.520558,11.114447,0,0,22.436038,2.509547,5.019095,22.436038,1.319767,3.792677,...,10.042771,50.615280,223.063329,8.579359,450,31,94.0,115.0,5.527778,3.666667
4,59.718429,48.210382,0,0,96.779823,2.445513,4.851677,96.779823,1.225061,5.262114,...,11.177523,134.866641,1111.641665,6.616915,30367,137,398.0,466.0,34.805556,17.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,18.849242,15.328468,0,0,31.713787,2.521082,4.896062,31.713787,1.378860,4.120187,...,10.205997,75.047243,322.100065,8.705407,1154,36,132.0,162.0,4.638889,4.916667
2334,29.852963,21.346319,0,1,49.974608,2.468887,4.896136,49.974608,1.315121,4.570280,...,10.549674,83.045006,521.311438,6.770278,5083,67,202.0,239.0,11.444444,8.361111
2335,17.165481,14.859018,0,0,27.856129,2.523211,5.005617,27.856129,1.266188,4.030589,...,10.154596,70.142462,302.115424,7.552886,980,37,118.0,142.0,8.000000,4.833333
2336,31.307617,21.998642,0,1,52.526684,2.449418,4.861108,52.526684,1.313167,4.616303,...,10.540038,85.147977,543.295788,7.055789,6018,68,210.0,246.0,11.944444,8.833333


In [ ]:
# Apply the function to all columns in the DataFrame
df2.isnull().sum()[df2.isnull().sum() == 2338]

SpAbs_Dt     2338
SpMax_Dt     2338
SpDiam_Dt    2338
SpAD_Dt      2338
SpMAD_Dt     2338
             ... 
MINssssSn    2338
MINsPbH3     2338
MINssPbH2    2338
MINsssPbH    2338
MINssssPb    2338
Length: 96, dtype: int64

In [ ]:
empty_cols = df2.isnull().sum()[df2.isnull().sum() == 2338].index
df2.drop(empty_cols, axis = 1, inplace = True)
df2

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,12.704061,10.489183,0,0,21.679248,2.500522,5.001045,21.679248,1.354953,3.734854,...,9.975203,49.287122,210.042927,9.547406,376,29,88.0,108.0,4.666667,3.500000
1,26.345939,21.039992,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,10.475737,87.611973,507.199695,7.803072,3400003745,57,180.0,216.0,NaN,7.583333
2,59.797706,41.739763,0,0,95.209049,2.573149,5.146298,95.209049,1.252751,5.260938,...,11.452400,116.888499,1084.472653,7.134689,32423,148,416.0,505.0,29.888889,16.333333
3,13.520558,11.114447,0,0,22.436038,2.509547,5.019095,22.436038,1.319767,3.792677,...,10.042771,50.615280,223.063329,8.579359,450,31,94.0,115.0,5.527778,3.666667
4,59.718429,48.210382,0,0,96.779823,2.445513,4.851677,96.779823,1.225061,5.262114,...,11.177523,134.866641,1111.641665,6.616915,30367,137,398.0,466.0,34.805556,17.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,18.849242,15.328468,0,0,31.713787,2.521082,4.896062,31.713787,1.378860,4.120187,...,10.205997,75.047243,322.100065,8.705407,1154,36,132.0,162.0,4.638889,4.916667
2334,29.852963,21.346319,0,1,49.974608,2.468887,4.896136,49.974608,1.315121,4.570280,...,10.549674,83.045006,521.311438,6.770278,5083,67,202.0,239.0,11.444444,8.361111
2335,17.165481,14.859018,0,0,27.856129,2.523211,5.005617,27.856129,1.266188,4.030589,...,10.154596,70.142462,302.115424,7.552886,980,37,118.0,142.0,8.000000,4.833333
2336,31.307617,21.998642,0,1,52.526684,2.449418,4.861108,52.526684,1.313167,4.616303,...,10.540038,85.147977,543.295788,7.055789,6018,68,210.0,246.0,11.944444,8.833333


In [ ]:
# df2.to_csv("df_mordred.csv")

In [ ]:
# # from google.colab import files
# # files.download("df_mordred.csv")

# # save directly to drive
# #!cp "df.mordred.csv" ""

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Importing datasets

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


url = "https://raw.githubusercontent.com/Ghaby-X/VP16-inhibitors-pred/main/Assayresults_preprocessed.csv"
url_mordred = "https://raw.githubusercontent.com/Ghaby-X/VP16-inhibitors-pred/main/df_mordred.csv"

In [24]:
df_mordred = pd.read_csv(url_mordred, index_col = 0)
df_raw = pd.read_csv(url)

In [25]:
df_mordred

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,12.704061,10.489183,0,0,21.679248,2.500522,5.001045,21.679248,1.354953,3.734854,...,9.975203,49.287122,210.042927,9.547406,376,29,88.0,108.0,4.666667,3.500000
1,26.345939,21.039992,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,10.475737,87.611973,507.199695,7.803072,3400003745,57,180.0,216.0,NaN,7.583333
2,59.797706,41.739763,0,0,95.209049,2.573149,5.146298,95.209049,1.252751,5.260938,...,11.452400,116.888499,1084.472653,7.134689,32423,148,416.0,505.0,29.888889,16.333333
3,13.520558,11.114447,0,0,22.436038,2.509547,5.019095,22.436038,1.319767,3.792677,...,10.042771,50.615280,223.063329,8.579359,450,31,94.0,115.0,5.527778,3.666667
4,59.718429,48.210382,0,0,96.779823,2.445513,4.851677,96.779823,1.225061,5.262114,...,11.177523,134.866641,1111.641665,6.616915,30367,137,398.0,466.0,34.805556,17.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,18.849242,15.328468,0,0,31.713787,2.521082,4.896062,31.713787,1.378860,4.120187,...,10.205997,75.047243,322.100065,8.705407,1154,36,132.0,162.0,4.638889,4.916667
2334,29.852963,21.346319,0,1,49.974608,2.468887,4.896136,49.974608,1.315121,4.570280,...,10.549674,83.045006,521.311438,6.770278,5083,67,202.0,239.0,11.444444,8.361111
2335,17.165481,14.859018,0,0,27.856129,2.523211,5.005617,27.856129,1.266188,4.030589,...,10.154596,70.142462,302.115424,7.552886,980,37,118.0,142.0,8.000000,4.833333
2336,31.307617,21.998642,0,1,52.526684,2.449418,4.861108,52.526684,1.313167,4.616303,...,10.540038,85.147977,543.295788,7.055789,6018,68,210.0,246.0,11.944444,8.833333


In [26]:
df_raw

,Unnamed: 0,PUBCHEM_CID,Canonical_smiles,Activity_score,Activity_outcome
0,0,72810.0,C1=CC2=C(C3=C(C=CC=N3)C(=O)C2=O)N=C1,100.0,1
1,1,23618032.0,CN(C)C1=NC=NC2=C1N=CN2[C@H]3[C@@H]([C@@H]([C@H...,99.0,1
2,2,99927.0,CC1C(C(CC(O1)OC2CC(OC(C2O)C)OC3=CC4=CC5=C(C(=O...,99.0,1
3,3,254021.0,C1=CC=C2C(=C1)C3=C(C=C(C=C3)N)C(=O)C2=O,99.0,1
4,4,45280821.0,CCC(C)C1C(CC(=O)O[C@H](C(=O)[C@H](C(=O)NC(C(=O...,99.0,1
...,...,...,...,...,...
2333,2337,665706.0,C1CC2=C(C1)NN=C2C3=NN=C4N3N=C(CS4)C5=CC=CC=C5,0.0,0
2334,2338,25015463.0,CN1CCC(CC1)NC(=O)C2=CC(=C(C=C2)NC3=NC=C4C(=N3)...,0.0,0
2335,2339,667208.0,CC1=CC2=C(C3=C(CCC3)C(=O)O2)C(=C1)OC(C)C(=O)OC,0.0,0
2336,2340,42628533.0,CN1CCC(CC1)NC(=O)C2=CC(=C(C=C2)NC3=NC=C4C(=N3)...,0.0,0


In [27]:
X = df_mordred
y = df_raw["Activity_outcome"]
Y = df_raw["Activity_score"]

## Splitting and Imputing preprocessor

 **Splitting the Dataset**

 The data set is splitted from the onset to prevent a look ahead bias

 "The purpose of Computation is *insight* not *numbers*" -Richard Hamming

In [28]:
#Splitting our data into train and test data using train_test_split module

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=15)

In [29]:
X_train

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
1925,20.482236,15.934713,0,0,35.071854,2.512657,4.881836,35.071854,1.348917,4.200445,...,10.183730,75.372284,349.179027,7.126103,1734,41,138.0,164.0,6.888889,5.750000
505,21.638832,16.085455,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,10.085851,63.992488,433.121149,8.020762,2800002438,41,142.0,162.0,NaN,6.166667
1493,12.158715,10.443317,0,0,20.516822,2.326846,4.653693,20.516822,1.282301,3.686090,...,9.437874,47.920298,215.069477,8.602779,447,21,78.0,88.0,4.944444,3.638889
733,16.527024,13.226854,0,0,27.003573,2.431030,4.830342,27.003573,1.285884,3.983162,...,10.010412,63.026880,301.113650,7.527841,914,37,112.0,132.0,6.618056,4.527778
320,18.050928,16.072118,0,0,29.856304,2.423571,4.767668,29.856304,1.244013,4.083346,...,10.006540,72.563259,330.121572,7.860037,1442,38,120.0,141.0,10.000000,5.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,19.965399,16.675758,0,0,32.963854,2.487935,4.886614,32.963854,1.267841,4.180573,...,10.123306,74.855427,374.162187,7.635963,1648,40,134.0,157.0,9.000000,5.861111
630,22.473356,16.373610,0,0,36.890704,2.357078,4.714157,36.890704,1.272093,4.281432,...,10.135512,64.228135,387.194677,7.170272,2812,42,148.0,168.0,9.840278,6.305556
2133,23.977329,18.100251,0,0,40.553762,2.489971,4.939152,40.553762,1.351792,4.353994,...,10.423144,79.659554,420.089226,9.132374,2529,50,164.0,197.0,7.611111,6.527778
1739,21.569882,15.657518,0,1,35.974599,2.309411,4.618822,35.974599,1.284807,4.243021,...,9.991544,62.780610,386.268176,6.230132,2581,39,140.0,158.0,8.888889,6.222222


**Handling missing values**

mean?, mode?, median?, interpolation?...

"https://www.sciencedirect.com/science/article/pii/S0895435618308710"

"To ask the right question is harder than to answer it" - George Cantor

In [30]:
# #All columns that contains missing values and the amount of missing values they contain
# X_train.isnull().sum()[X_train.isnull().sum() != 0 ]

In [31]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Plotting a KDE plot
# sns.kdeplot(X_train['SpAbs_A'])
# plt.xlabel('SpAbs_A')
# plt.ylabel('Density')
# plt.title('KDE Plot of SpAbs_A')
# plt.show()

In [32]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy = "mean")
# imputer.fit(X_train)
# X_train = pd.DataFrame(imputer.transform(X_train), columns = X_train.columns)
# X_train

In [33]:
# # Plotting a KDE plot
# sns.kdeplot(X_train['SpAbs_A'])
# plt.xlabel('SpAbs_A')
# plt.ylabel('Density')
# plt.title('KDE Plot of SpAbs_A')
# plt.show()

In [34]:
# X_train.isnull().sum()[X_train.isnull().sum() != 0]

In [35]:
# Preprocessing pipeline

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# from sklearn.impute import KNNImputer


preprocessor = Pipeline([
              ("Imputation", SimpleImputer(strategy = "mean"))
])

preprocessor.fit(X_train)



Pipeline(steps=[('Imputation', SimpleImputer())])

In [36]:
X_train = pd.DataFrame(preprocessor.transform(X_train), columns = X_train.columns)
X_train

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,20.482236,15.934713,0.0,0.0,35.071854,2.512657,4.881836,35.071854,1.348917,4.200445,...,10.183730,75.372284,349.179027,7.126103,1.734000e+03,41.0,138.0,164.0,6.888889,5.750000
1,21.638832,16.085455,0.0,1.0,33.724737,2.443935,4.831111,33.724737,1.286329,4.164896,...,10.085851,63.992488,433.121149,8.020762,2.800002e+09,41.0,142.0,162.0,8.747166,6.166667
2,12.158715,10.443317,0.0,0.0,20.516822,2.326846,4.653693,20.516822,1.282301,3.686090,...,9.437874,47.920298,215.069477,8.602779,4.470000e+02,21.0,78.0,88.0,4.944444,3.638889
3,16.527024,13.226854,0.0,0.0,27.003573,2.431030,4.830342,27.003573,1.285884,3.983162,...,10.010412,63.026880,301.113650,7.527841,9.140000e+02,37.0,112.0,132.0,6.618056,4.527778
4,18.050928,16.072118,0.0,0.0,29.856304,2.423571,4.767668,29.856304,1.244013,4.083346,...,10.006540,72.563259,330.121572,7.860037,1.442000e+03,38.0,120.0,141.0,10.000000,5.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,19.965399,16.675758,0.0,0.0,32.963854,2.487935,4.886614,32.963854,1.267841,4.180573,...,10.123306,74.855427,374.162187,7.635963,1.648000e+03,40.0,134.0,157.0,9.000000,5.861111
1632,22.473356,16.373610,0.0,0.0,36.890704,2.357078,4.714157,36.890704,1.272093,4.281432,...,10.135512,64.228135,387.194677,7.170272,2.812000e+03,42.0,148.0,168.0,9.840278,6.305556
1633,23.977329,18.100251,0.0,0.0,40.553762,2.489971,4.939152,40.553762,1.351792,4.353994,...,10.423144,79.659554,420.089226,9.132374,2.529000e+03,50.0,164.0,197.0,7.611111,6.527778
1634,21.569882,15.657518,0.0,1.0,35.974599,2.309411,4.618822,35.974599,1.284807,4.243021,...,9.991544,62.780610,386.268176,6.230132,2.581000e+03,39.0,140.0,158.0,8.888889,6.222222


## **Feature Selection**

**Goal** : To select the most statistically relevant columns


Article link: "https://www.sciencedirect.com/science/article/pii/S0016236122006950#:~:text=The%20method%20focuses%20on%20reducing%20the%20number%20of,identify%20dominant%20chemical-structure%20features%20that%20impact%20property%20values."

In [37]:
#removing low_variance columns

from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.1)
selector.fit(X_train)

mask = selector.get_support()
high_var_cols = X_train.columns[mask]

X_train_var= X_train[high_var_cols].copy()
X_train_var

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpAD_A,VE1_A,VR1_A,VR2_A,VR3_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,20.482236,15.934713,0.0,0.0,35.071854,35.071854,3.967264,578.334610,22.243639,7.315664,...,10.183730,75.372284,349.179027,7.126103,1.734000e+03,41.0,138.0,164.0,6.888889,5.750000
1,21.638832,16.085455,0.0,1.0,33.724737,33.724737,4.152498,5170.655227,110.043193,6.769233,...,10.085851,63.992488,433.121149,8.020762,2.800002e+09,41.0,142.0,162.0,8.747166,6.166667
2,12.158715,10.443317,0.0,0.0,20.516822,20.516822,3.633555,83.141235,5.196327,4.890544,...,9.437874,47.920298,215.069477,8.602779,4.470000e+02,21.0,78.0,88.0,4.944444,3.638889
3,16.527024,13.226854,0.0,0.0,27.003573,27.003573,4.121754,130.897651,6.233221,5.616353,...,10.010412,63.026880,301.113650,7.527841,9.140000e+02,37.0,112.0,132.0,6.618056,4.527778
4,18.050928,16.072118,0.0,0.0,29.856304,29.856304,3.823930,256.509849,10.687910,6.422636,...,10.006540,72.563259,330.121572,7.860037,1.442000e+03,38.0,120.0,141.0,10.000000,5.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,19.965399,16.675758,0.0,0.0,32.963854,32.963854,4.047858,305.745452,11.759440,6.678264,...,10.123306,74.855427,374.162187,7.635963,1.648000e+03,40.0,134.0,157.0,9.000000,5.861111
1632,22.473356,16.373610,0.0,0.0,36.890704,36.890704,4.452620,475.386892,16.392651,7.228840,...,10.135512,64.228135,387.194677,7.170272,2.812000e+03,42.0,148.0,168.0,9.840278,6.305556
1633,23.977329,18.100251,0.0,0.0,40.553762,40.553762,4.193960,582.454456,19.415149,7.465863,...,10.423144,79.659554,420.089226,9.132374,2.529000e+03,50.0,164.0,197.0,7.611111,6.527778
1634,21.569882,15.657518,0.0,1.0,35.974599,35.974599,4.955499,179.262120,6.402219,6.218469,...,9.991544,62.780610,386.268176,6.230132,2.581000e+03,39.0,140.0,158.0,8.888889,6.222222


In [38]:
def correlation(dataset, threshold):
  col_corr = set()  #Set of all the names of correlated columns
  corr_matrix = dataset.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
      if abs(corr_matrix.iloc[i,j]) > threshold:
        colname = corr_matrix.columns[i]
        col_corr.add(colname)
  return col_corr

In [39]:
corr_features = correlation(X_train_var, 0.9)
col_names = list(X_train_var.drop(corr_features, axis = 1).columns)
len(col_names)

346

In [40]:
X_train = X_train[col_names].copy()
X_train

,ABC,nAcid,nBase,VE1_A,VR1_A,VR3_A,nAromAtom,nBridgehead,nHetero,nN,...,n10FaRing,nFARing,nG12FARing,nRot,SLogP,TopoPSA(NO),SRW05,TSRW10,AMW,WPath
0,20.482236,0.0,0.0,3.967264,578.334610,7.315664,12.0,0.0,5.0,3.0,...,0.0,1.0,0.0,4.0,3.30010,62.30,2.397895,75.372284,7.126103,1.734000e+03
1,21.638832,0.0,1.0,4.152498,5170.655227,6.769233,12.0,0.0,6.0,1.0,...,0.0,0.0,0.0,5.0,5.29520,46.61,0.000000,63.992488,8.020762,2.800002e+09
2,12.158715,0.0,0.0,3.633555,83.141235,4.890544,12.0,0.0,5.0,3.0,...,0.0,0.0,0.0,3.0,2.73340,68.06,0.000000,47.920298,8.602779,4.470000e+02
3,16.527024,0.0,0.0,4.121754,130.897651,5.616353,12.0,0.0,4.0,1.0,...,0.0,1.0,0.0,2.0,2.78452,37.38,0.000000,63.026880,7.527841,9.140000e+02
4,18.050928,0.0,0.0,3.823930,256.509849,6.422636,11.0,0.0,7.0,2.0,...,0.0,0.0,0.0,6.0,3.07274,91.44,2.397895,72.563259,7.860037,1.442000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,19.965399,0.0,0.0,4.047858,305.745452,6.678264,15.0,0.0,8.0,6.0,...,0.0,0.0,0.0,7.0,3.98700,87.89,2.397895,74.855427,7.635963,1.648000e+03
1632,22.473356,0.0,0.0,4.452620,475.386892,7.228840,18.0,0.0,5.0,3.0,...,0.0,0.0,0.0,5.0,4.56140,71.09,0.000000,64.228135,7.170272,2.812000e+03
1633,23.977329,0.0,0.0,4.193960,582.454456,7.465863,21.0,0.0,9.0,4.0,...,0.0,1.0,0.0,5.0,2.85500,99.10,2.397895,79.659554,9.132374,2.529000e+03
1634,21.569882,0.0,1.0,4.955499,179.262120,6.218469,6.0,0.0,6.0,4.0,...,0.0,0.0,0.0,6.0,2.85200,64.68,0.000000,62.780610,6.230132,2.581000e+03


# **Data Imbalance**

In [41]:
print(f"Total number of compounds: {y_train.count()} \nTotal actives: {y_train[y_train == 1].count()} \nTotal inactives: {y_train[y_train == 0].count()}")

Total number of compounds: 1636 
Total actives: 695 
Total inactives: 941


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(5.5, 5.5))

sns.countplot(x ="Activity_outcome" , data=df_raw, edgecolor='black', palette=['green', 'yellow'])

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

#plt.savefig('plot_bioactivity_class.pdf')
plt.show()


In [42]:
#resampling our data using SMOTE

from imblearn.over_sampling import SMOTE
# assume X is your feature matrix and y is your target vector
smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [43]:
print(f"Total number of compounds: {y_resampled.count()} \nTotal actives: {y_resampled[y_resampled == 1].count()} \nTotal inactives: {y_resampled[y_resampled == 0].count()}")

Total number of compounds: 1882 
Total actives: 941 
Total inactives: 941


In [ ]:
# plt.figure(figsize=(5.5, 5.5))

# sns.countplot(x =y_resampled , edgecolor='black', palette=['green', 'yellow'])

# plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
# plt.ylabel('Frequency', fontsize=14, fontweight='bold')

# #plt.savefig('plot_bioactivity_class.pdf')
# plt.show()


In [44]:
#Recursive feature elimination
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(random_state = 42)

rfe = RFECV(estimator, cv = 5, n_jobs = -1)
rfe.fit(X_resampled, y_resampled)

RFECV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1)

In [51]:
rfe_mask = rfe.get_support()
X_resampled = X_resampled[X_resampled.columns[rfe_mask]]

In [54]:
#preprocessing the X_test

X_test = pd.DataFrame(preprocessor.transform(X_test), columns = X_test.columns)
X_test = X_test[col_names]
X_test = X_test[X_test.columns[rfe_mask]]

In [55]:

rfc = RandomForestClassifier(random_state = 42)
rfc.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=42)

In [58]:
evaluate(X_test, y_test, rfc)

,scores
accuracy,0.820513
sensitivity,0.742268
specificity,0.875912
precision,0.808989
F1,0.774194
balanced_accuracy,0.809090
MCC,0.627310
auc_roc,0.891656


In [64]:
from sklearn.model_selection import GridSearchCV

estimator = RandomForestClassifier(random_state = 42)
params_grid = {
    "n_estimators": [100,300,500,700,1000],
    "criterion": ["gini", "entropy", "log_loss"],
    "max_features": ["sqrt", "log2", None]

}

g_search = GridSearchCV(estimator, params_grid, cv = 5)
g_search.fit(X_resampled, y_resampled)

KeyboardInterrupt: ignored

# Applicability Domain Analysis using standardization Approach

Article: https://www.sciencedirect.com/science/article/pii/S0169743915000969

In [33]:
from sklearn.preprocessing import StandardScaler

def standardized_similarity(X_tain, X_test):
  scaler = StandardScaler()
  scaler.fit(X_train)


  mean_htrain = np.mean(scaler.transform(X_train), axis=1)
  std_htrain = np.std(scaler.transform(X_train), axis=1)

  s_new_train = (mean_htrain + 1.28 * std_htrain)


  # for test data
  mean_htest = np.mean(scaler.transform(X_test), axis = 1)
  std_htest = np.std(scaler.transform(X_test), axis = 1)

  s_new_test = (mean_htest + 1.28 * std_htest)
  return [s_new_train, s_new_test]

def visualize_AD(s_new_train, s_new_test):
  plt.plot(s_new_train, "bo", color = "b")
  plt.plot(s_new_test, "bo", color = "g")
  plt.axhline(y = 3, color='r', linestyle='-', linewidth = 2.0)  #plots a horizontal line to denote the max and min y threshold


  plt.xlabel("Compound ID")
  plt.ylabel("Leverage")
  plt.title("Applicability Domain")
  plt.show()


def isWithinAD(X_train, X_test):
  scaler = StandardScaler()
  scaler.fit(X_train)

  AD_identity = []

  mean_htest = np.mean(scaler.transform(X_test), axis = 1)
  std_htest = np.std(scaler.transform(X_test), axis = 1)

  s_new_test = (mean_htest + 1.28 * std_htest)


  return s_new_test < 3

In [37]:
AD_mask = isWithinAD(X_resampled, X_test)

In [40]:
X_test_inliers = X_test[AD_mask]
y_test_inliers = y_test[AD_mask]

In [41]:
evaluate(X_test_inliers, y_test_inliers, rfc)

,scores
accuracy,0.807471
sensitivity,0.743860
specificity,0.851582
precision,0.776557
F1,0.759857
balanced_accuracy,0.797721
MCC,0.599695
auc_roc,0.889166


In [43]:
#scaling the dataset

scaley = StandardScaler()
scaley.fit(X_resampled)

X_strain = scaley.transform(X_resampled)
X_stest = scaley.transform(X_test)

rfc.fit(X_strain, y_resampled)

RandomForestClassifier(random_state=42)

In [44]:
evaluate(X_stest, y_test, rfc)

,scores
accuracy,0.809117
sensitivity,0.738832
specificity,0.858881
precision,0.787546
F1,0.762411
balanced_accuracy,0.798856
MCC,0.604018
auc_roc,0.889248


# preprocessing pipeline

In [ ]:
#Splitting our data into train and test data using train_test_split module

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=15)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier


Column_selector = ColumnTransformer(
    transformers=[
        ("relevant_cols", "passthrough", col_names)
    ]
)

preprocess_pipeline = Pipeline([
              ("preprocessor", Column_selector),
              ("Imputation", SimpleImputer(strategy = "mean")),
              ("Removing low variance features", VarianceThreshold(threshold = 0.1)),
              ("Scaling", StandardScaler()),
              #("RFE", RFE(RandomForestClassifier(n_estimators = 1000, random_state = 42)))
])


# define pipeline for handling dataImbalance
from imblearn.pipeline import Pipeline as imb_Pipeline

sm_pipeline = imb_Pipeline([
           ("Data Imbalance", SMOTE(random_state = 42, k_neighbors = 5))
])

preprocess_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('relevant_cols',
                                                  'passthrough',
                                                  ['ABC', 'nAcid', 'nBase',
                                                   'VE1_A', 'VR1_A', 'VR3_A',
                                                   'nAromAtom', 'nBridgehead',
                                                   'nHetero', 'nN', 'nO', 'nS',
                                                   'nF', 'nCl', 'nX', 'ATS0dv',
                                                   'ATS0s', 'ATS3s', 'ATS0Z',
                                                   'ATS0p', 'AATS0dv',
                                                   'AATS7dv', 'AATS8dv',
                                                   'AATS0d', 'AATS3d', 'AATS4d',
                                                   'AATS5d', 'AATS6d', 'AATS7d',
                                                   'AATS8d', ...])])),
                ('Imputation', SimpleImputer()),
                ('Removing low variance features',
                 VarianceThreshold(threshold=0.1)),
                ('Scaling', StandardScaler())])

In [ ]:
#transforming the datasets
transformed_X_train = preprocess_pipeline.transform(X_train)
transformed_X_test = preprocess_pipeline.transform(X_test)

#adding the smote pipeline to handle the data Imbalance in transformed X_train
X_train_resampled, y_train_resampled = sm_pipeline.fit_resample(transformed_X_train, y_train)

In [ ]:
pd.DataFrame(X_train_resampled, columns = col_names)

,ABC,nAcid,nBase,VE1_A,VR1_A,VR3_A,nAromAtom,nBridgehead,nHetero,nN,...,n10FaRing,nFARing,nG12FARing,nRot,SLogP,TopoPSA(NO),SRW05,TSRW10,AMW,WPath
0,0.021411,-0.308161,-0.431974,-4.639880e-01,-0.054636,4.492980e-01,-0.293783,-0.124799,-0.798566,0.064162,...,-0.353999,1.525545,-0.350714,-0.296988,0.186942,-0.468134,0.591448,0.449200,-1.037296,-0.222681
1,0.237495,-0.308161,1.472904,-2.224782e-15,0.000000,-7.302970e-16,-0.293783,-0.124799,-0.419884,-1.208181,...,-0.353999,-0.584169,-0.350714,0.104892,1.616609,-0.895827,-1.202457,-0.535719,-0.273977,0.726120
2,-1.533653,-0.308161,-0.431974,-1.299890e+00,-0.060528,-1.544736e+00,-0.293783,-0.124799,-0.798566,0.064162,...,-0.353999,-0.584169,-0.350714,-0.698869,-0.219149,-0.311123,-1.202457,-1.926763,0.222597,-0.222681
3,-0.717532,-0.308161,-0.431974,-7.700849e-02,-0.059959,-9.479458e-01,-0.293783,-0.124799,-1.177248,-1.208181,...,-0.353999,1.525545,-0.350714,-1.100749,-0.182517,-1.147426,-1.202457,-0.619292,-0.694535,-0.222681
4,-0.432825,-0.308161,-0.431974,-8.230242e-01,-0.058465,-2.849868e-01,-0.482338,-0.124799,-0.041201,-0.572010,...,-0.353999,-0.584169,-0.350714,0.506772,0.024019,0.326190,0.591448,0.206080,-0.411107,-0.222681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,-0.167280,-0.308161,0.043074,-4.408268e-01,-0.042863,6.303343e-02,-0.717453,-0.124799,-0.419884,-1.208181,...,-0.353999,-0.058037,0.422742,-0.094566,-0.339340,-0.152760,1.075201,0.475498,-0.730625,-0.045218
1878,-0.550702,-0.308161,-0.431974,7.863544e-02,-0.059417,-7.543580e-01,-0.220938,-0.124799,0.789311,1.336505,...,-0.353999,-0.584169,-0.350714,-0.867861,-2.984416,1.157590,1.366566,0.302139,0.138601,-0.222681
1879,0.191947,-0.308161,-0.413888,5.839798e-01,-0.058380,-1.928932e-01,1.027897,-0.124799,0.719758,1.342545,...,-0.353999,-0.584169,-0.350714,-0.687421,-0.198874,1.267340,0.574416,0.582833,0.617056,-0.222681
1880,-0.052050,-0.308161,-0.431974,3.807069e-01,-0.058422,-2.045525e-01,0.460440,-0.124799,-0.954817,-0.198334,...,2.726548,-0.584169,-0.350714,0.340949,0.768832,-0.418438,-1.202457,-0.784917,-1.113937,-0.222681


In [ ]:
from sklearn.feature_selection import RFECV

estimator  = RandomForestClassifier(random_state = 140)
rfe = RFECV(estimator, step = 1, cv = 5)

rfe.fit(pd.DataFrame(X_train_resampled, columns = col_names), y_train_resampled)

RFECV(cv=5, estimator=RandomForestClassifier(random_state=140))

# Evaluation metrics

In [57]:
def evaluate(X_test, y_test, model, title="model"):
    from sklearn.metrics import matthews_corrcoef
    from sklearn.metrics import roc_auc_score
    import pandas as pd

    y_pred = model.predict(X_test)
    pred_proba = model.predict_proba(X_test)[:, 1]

    TP = sum((y_pred == 1) & (y_test == 1))
    TN = sum((y_pred == 0) & (y_test == 0))
    FP = sum((y_pred == 1) & (y_test == 0))
    FN = sum((y_pred == 0) & (y_test == 1))

    accuracy = (TP + TN) / (TP + TN + FN + FP)
    sensitivity = (TP) / (TP + FN)
    specificity = (TN) / (TN + FP)
    precision = (TP) / (TP + FP)
    F1 = 2 * ((precision * sensitivity) / (precision + sensitivity))
    balanced_accuracy = (sensitivity + specificity) / 2
    mcc = matthews_corrcoef(y_test, y_pred)
    AUCROC = roc_auc_score(y_test, pred_proba)

    values = [accuracy, sensitivity, specificity, precision, F1, balanced_accuracy, mcc, AUCROC]
    index = ["accuracy", "sensitivity", "specificity", "precision", "F1", "balanced_accuracy", "MCC", "auc_roc"]

    return pd.DataFrame({"scores": values}, index=index)


def plot_roc(X_test, y_test, model):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, roc_auc_score

    y_pred = model.predict(X_test)
    pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate the false positive rate (fpr), true positive rate (tpr), and thresholds
    fpr, tpr, thresholds = roc_curve(y_test, pred_proba)

    # Calculate the AUC
    roc_auc = roc_auc_score(y_test, pred_proba)

    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, labe0l='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier


# Define and train the model
rfc = RandomForestClassifier(n_estimators = 700, random_state = 140)
rfc.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(n_estimators=700, random_state=140)

In [ ]:
evaluate(transformed_X_test, y_test, rfc)

,scores
accuracy,0.821937
sensitivity,0.738832
specificity,0.880779
precision,0.814394
F1,0.774775
balanced_accuracy,0.809805
MCC,0.630154
auc_roc,0.899386


In [ ]:
evaluate(transformed_X_test, y_test, rfc)

,scores
accuracy,0.833333
sensitivity,0.769759
specificity,0.878345
precision,0.817518
F1,0.792920
balanced_accuracy,0.824052
MCC,0.654509
auc_roc,0.899478


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train_resampled)


xgb = GradientBoostingClassifier(n_estimators = 700, learning_rate = 0.1, random_state = 1)
xgb.fit(scaler.transform(X_train_resampled), y_train_resampled)

GradientBoostingClassifier(n_estimators=700, random_state=1)

In [ ]:
evaluate(transformed_X_test, y_test, xgb)

,scores
accuracy,0.814815
sensitivity,0.745704
specificity,0.863747
precision,0.794872
F1,0.769504
balanced_accuracy,0.804726
MCC,0.615881
auc_roc,0.891782


In [ ]:
from sklearn.svm import SVC


svc = SVC(kernel = "poly", degree =3, coef0 =1.0, probability = True)
svc.fit(scaler.transform(X_train_resampled), y_train_resampled)

SVC(coef0=1.0, kernel='poly', probability=True)

In [ ]:
evaluate(scaler.transform(transformed_X_test), y_test, svc)

,scores
accuracy,0.824786
sensitivity,0.783505
specificity,0.854015
precision,0.791667
F1,0.787565
balanced_accuracy,0.818760
MCC,0.638505
auc_roc,0.880544


In [ ]:
#knn
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit()